In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_train_est_A = pd.read_csv('train_est_C.csv')
df_train_obs_A = pd.read_csv('train_obs_C.csv')
df_test_A = pd.read_csv('X_test_C.csv')
df_train_est_A = df_train_est_A.rename(columns={'Unnamed: 0': 'date_forecast'})

df_train_obs_A = df_train_obs_A.rename(columns={'Unnamed: 0': 'date_forecast'})

df_test_A = df_test_A.rename(columns={'Unnamed: 0': 'date_forecast'})

df_train_est_A.set_index('date_forecast', inplace=True)
df_train_obs_A.set_index('date_forecast', inplace=True)
df_test_A.set_index('date_forecast', inplace=True)
df_train_obs_A.index = pd.to_datetime(df_train_obs_A.index)
df_train_obs_A.index = pd.to_datetime(df_train_obs_A.index)

In [3]:
df = pd.concat([df_train_obs_A,df_train_est_A])
df = df.drop(columns = 'date_calc', axis = 1)
df_test_A = df_test_A.drop(columns = 'date_calc', axis = 1)

In [29]:
import re
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [37]:
df_nandrop = df
df_nandrop = df_nandrop.drop(columns= ['snow_densitykgm3'],axis = 1)
df_nandrop = df_nandrop.dropna()

df_nan0 = df.fillna(0)
df_nanmedian = df.fillna(df.median())

In [38]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
# Imputa i valori mancanti
df_knn =pd.DataFrame(imputer.fit_transform(df),columns = df.columns, index = df.index)

In [39]:
df_dict = {
    'drop' : df_nandrop,
    'zero' : df_nan0,
    'median' : df_nanmedian,
    'knn' : df_knn,
}

In [40]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_absolute_error
# Define the number of splits (e.g., 5 for 80-20 train-test splits)
n_splits = 3
# Initialize the TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=n_splits)

In [41]:
import lightgbm as lgb
param = {'verbose' :-1}

model = lgb.LGBMRegressor(**param)

In [42]:
def test(df_dict):
    mean_mae_list = []
    for key in df_dict:
        print(key)
        data = df_dict[key]
        maelist = []
        for train_index, test_index in tscv.split(data):
            train_data = data.iloc[train_index]
            test_data = data.iloc[test_index]

            # Extract target variable for training and testing data
            y_train = train_data['pv_measurement']
            y_test = test_data['pv_measurement']

            # Extract features for training and testing data
            X_train = train_data.drop(columns = 'pv_measurement')
            X_test = test_data.drop(columns = 'pv_measurement')

            # Train the XGBoost model
            model.fit(X_train, y_train,eval_set=[(X_test,y_test)])

            # Make predictions on the test data
            y_pred = model.predict(X_test)

            # Evaluate the model using Mean Absolute Error (MAE)
            mae = mean_absolute_error(y_test, y_pred)
            print("Mean Absolute Error:", mae)
            maelist.append(mae)
        # Return MAE
        mean_mae = np.average(maelist,weights=[1,2,3])
        print("Mean MAE: ",mean_mae )
        mean_mae_list.append(mean_mae)
    return mean_mae_list

In [43]:
test(df_dict)

drop
Mean Absolute Error: 39.24107551252952
Mean Absolute Error: 37.346138506512176
Mean Absolute Error: 14.490016527606235
Mean MAE:  26.23390035139543
zero
Mean Absolute Error: 98.79672811531096
Mean Absolute Error: 34.9474371903615
Mean Absolute Error: 18.375115282463135
Mean MAE:  37.302824723903896
median
Mean Absolute Error: 98.85084688972952
Mean Absolute Error: 35.39362977321388
Mean Absolute Error: 17.903465495469998
Mean MAE:  37.224750487094546
knn
Mean Absolute Error: 34.95919711518043
Mean Absolute Error: 29.098108001739494
Mean Absolute Error: 16.65688442748432
Mean MAE:  23.854344400185397


[26.23390035139543, 37.302824723903896, 37.224750487094546, 23.854344400185397]

In [51]:
columns = df_nandrop.drop(columns = 'pv_measurement').columns


In [52]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Crea un oggetto MinMaxScaler
scaler = MinMaxScaler()

# Normalizza il DataFrame
df_normalized = pd.DataFrame(scaler.fit_transform(df_knn[columns]),columns = df_knn[columns].columns, index = df_knn[columns].index)
df_normalized['pv_measurement'] = df_knn[ 'pv_measurement']

In [53]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
# Crea un oggetto StandardScaler
scaler = StandardScaler()

# Standardizza il DataFrame
df_scaled = pd.DataFrame(scaler.fit_transform(df_knn[columns]),columns = df_knn[columns].columns, index = df_knn[columns].index)
df_scaled['pv_measurement'] = df_knn[ 'pv_measurement']

In [54]:
def z_score_standardization(data):
  mean = np.mean(data)
  std = np.std(data)
  standardized_data = (data - mean) / std
  return standardized_data

def z_score_standardization_dataframe(df):
    standardized_df = pd.DataFrame()
    for column in df.columns:
        standardized_df[column] = z_score_standardization(df[column])
    return standardized_df

# Esempio di utilizzo della funzione:

df_zscore = df_knn[columns]

df_zscore = z_score_standardization_dataframe(df_zscore)
df_zscore['pv_measurement'] = df_knn[ 'pv_measurement']

In [55]:
df_scaled_norm = {
    'norm' : df_normalized,
    'scaled' : df_scaled,
    'zscore' : df_zscore,
}

In [56]:
test(df_scaled_norm)

norm
Mean Absolute Error: 34.90863321039037
Mean Absolute Error: 29.122741607412955
Mean Absolute Error: 16.46745247464237
Mean MAE:  23.759412308190566
scaled
Mean Absolute Error: 35.15013672830231
Mean Absolute Error: 29.132501168081177
Mean Absolute Error: 16.5837881034236
Mean MAE:  23.86108389578924
zscore
Mean Absolute Error: 35.15013672830231
Mean Absolute Error: 29.132501168081177
Mean Absolute Error: 16.5837881034236
Mean MAE:  23.86108389578924


[23.759412308190566, 23.86108389578924, 23.86108389578924]

In [ ]:
df_normalized['direct_radW'].plot()

In [ ]:
df_scaled['direct_radW'].plot()

In [ ]:
df_zscore['direct_radW'].plot()